In [ ]:
import json
import subprocess
import time

In [ ]:
# query anchors from the city list.

anchors_by_city = {}

with open('../../info/30-largest-cities-by-population-2026.json', 'r') as file_cities:
    data = json.load(file_cities)
    for entry in data:
        anchors = []

        city = entry['city']
        city_encoded = city.replace(' ', '%20')
        url = f'https://atlas.ripe.net/api/v2/anchors?search={city_encoded}'

        cp = subprocess.run(
            ['curl', '-s', url],
            capture_output=True,
            text=True,
            check=True
        )
        curl_result = json.loads(cp.stdout.strip())
        for anchor_entry in curl_result['results']:
            if not 'ip_v4' in anchor_entry:
                continue
            if anchor_entry['ip_v4'] in [None, '']:
                continue
            if anchor_entry['is_disabled'] == True:
                continue
            if anchor_entry['date_decommissioned'] != None:
                continue

            anchors.append(anchor_entry)    

        anchors_by_city[city] = anchors
        print(f'{city}: {len(anchors)}')


In [42]:
city_by_anchor_probe = {}

for city, anchors in anchors_by_city.items():
    for anchor in anchors:
        city_by_anchor_probe[anchor['probe']] = city

In [37]:
city_measurements = {} #dict[city]dict[fqdn]measurement

for city, anchors in anchors_by_city.items():
    anchor_measurements = {} #key: fqdn

    for anchor in anchors:
        measurements = []

        #print(anchor)
        url = f'https://atlas.ripe.net/api/v2/measurements/?target={anchor['fqdn']}&tags=anchoring,mesh&type=ping&status=Ongoing&is_public=True'
        cp = subprocess.run(
            ['curl', '-s', url],
            capture_output=True,
            text=True,
            check=True
        )
        curl_result = json.loads(cp.stdout.strip())

        for measurement in curl_result['results']:
            if ':' in measurement['target_ip']: #ipv6
                continue

            measurements.append(measurement)

        anchor_measurements[anchor['fqdn']] = measurements
        print(f'{anchor['fqdn']}: {len(measurements)}')
    
    city_measurements[city] = anchor_measurements


jp-tyo-as141682-client.anchors.atlas.ripe.net: 1
jp-tyo-as142616.anchors.atlas.ripe.net: 1
jp-tyo-as15133.anchors.atlas.ripe.net: 2
jp-tyo-as151420.anchors.atlas.ripe.net: 1
jp-tyo-as199524.anchors.atlas.ripe.net: 1
jp-tyo-as20473.anchors.atlas.ripe.net: 1
jp-tyo-as2497.anchors.atlas.ripe.net: 1
jp-tyo-as3491.anchors.atlas.ripe.net: 1
jp-tyo-as38649.anchors.atlas.ripe.net: 1
jp-tyo-as45679.anchors.atlas.ripe.net: 0
jp-tyo-as49544.anchors.atlas.ripe.net: 1
jp-tyo-as55569.anchors.atlas.ripe.net: 1
jp-tyo-as59105.anchors.atlas.ripe.net: 1
jp-tyo-as63781.anchors.atlas.ripe.net: 1
jp-tyo-as63834.anchors.atlas.ripe.net: 1
id-jkt-as133800.anchors.atlas.ripe.net: 1
id-jkt-as141640.anchors.atlas.ripe.net: 1
id-jkt-as153112.anchors.atlas.ripe.net: 1
id-jkt-as24523.anchors.atlas.ripe.net: 1
id-jkt-as45701.anchors.atlas.ripe.net: 1
id-jkt-as45725.anchors.atlas.ripe.net: 1
id-jkt-as64300.anchors.atlas.ripe.net: 1
in-del-as12008.anchors.atlas.ripe.net: 1
in-bom-as12008.anchors.atlas.ripe.net: 1
in-n

In [73]:
time_stop = int(time.time())
time_start = time_stop - 5 * 60 * 60 #an hour
print(f'start:{time_start}, stop:{time_stop}')

start:1770548078, stop:1770566078


In [70]:
# for each measurement, we query every other city anchor's probe
all_probes_str = ','.join(map(str, city_by_anchor_probe.keys()))

In [ ]:
city_ping_results = {} # dict['city|target']dict['probe']list[rtt]

for city, city_entry in city_measurements.items():
    for fqdn, measurements in city_entry.items():
        for measurement in measurements:
            city_msm_id = f'{city}|{measurement['target']}'
            print(city_msm_id)

            ping_results = {}
            for probe_id in city_by_anchor_probe.keys():
                ping_results[probe_id] = []

            url = f'{measurement['result']}?probe_ids={all_probes_str}&start={time_start}&stop={time_stop}&format=json'
            cp = subprocess.run(
                ['curl', '-s', url],
                capture_output=True,
                text=True,
                check=True
            )
            curl_result = json.loads(cp.stdout.strip())

            for result_entry in curl_result:
                #print(result_entry)
                ping_results[result_entry['prb_id']].extend(result_entry['result'])

            city_ping_results[city_msm_id] = ping_results
        break
    break

Tokyo|jp-tyo-as141682-client.anchors.atlas.ripe.net


{'Tokyo|jp-tyo-as141682-client.anchors.atlas.ripe.net': {7663: [], 7039: [], 6367: [], 7270: [], 6631: [], 6455: [], 6425: [], 7080: [], 7689: [], 7563: [], 7642: [], 6974: [], 6370: [], 7074: [], 6580: [], 6681: [], 7441: [], 7423: [], 7252: [], 7284: [], 7275: [], 7573: [], 6953: [], 6954: [], 7674: [], 7622: [], 6977: [], 6659: [], 6641: [], 6792: [], 7675: [], 7172: [], 6420: [], 6972: [], 6783: [], 7065: [], 6785: [], 6905: [], 7165: [], 7168: [], 6583: [], 6837: [], 6338: [], 6335: [], 7140: [], 7424: [], 6491: [], 6733: [], 7505: [], 7578: [], 7623: [], 6989: [], 7516: [], 7518: [], 6649: [], 6598: [], 6492: [], 7514: [], 7360: [], 7517: [], 6811: [], 7585: [], 7614: [], 7330: [], 6718: [], 7400: [], 7515: [], 7560: [], 7558: [], 6993: [], 7362: [], 6779: [], 7063: [], 7222: [], 6569: [], 6935: [], 6344: [], 6662: [], 7373: [], 7167: [], 7378: [], 7438: [], 6429: [], 6776: [], 6318: [], 7648: [], 7644: [], 6582: [], 6786: [], 7592: [], 7027: [], 7269: [], 7613: [], 6945: [], 697